# HybridValues

<a href="https://colab.research.google.com/github/borglab/gtsam/blob/develop/gtsam/hybrid/doc/HybridValues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    import google.colab
    %pip install --quiet gtsam-develop
except ImportError:
    pass  # Not running on Colab, do nothing

`HybridValues` is a container class in GTSAM designed to hold results from hybrid inference. It stores three types of variable assignments simultaneously:

1.  **Continuous `VectorValues`**: Stores vector-valued assignments for continuous variables, typically used with Gaussian factors/conditionals (`gtsam.GaussianFactor`, `gtsam.GaussianConditional`).
2.  **Discrete `DiscreteValues`**: Stores assignments (unsigned integers) for discrete variables (`gtsam.DiscreteKey`, `gtsam.DiscreteFactor`).
3.  **Nonlinear `Values`**: Stores assignments for variables living on manifolds, used with nonlinear factors (`gtsam.NonlinearFactor`).

It provides a unified way to represent the complete state (or solution) in a hybrid system.

In [2]:
import gtsam
import numpy as np

from gtsam import (
    HybridValues, VectorValues, DiscreteValues, Values, Pose2
)
from gtsam.symbol_shorthand import X, D

## Initialization

In [3]:
# 1. Default constructor (empty)
hybrid_values_empty = HybridValues()
print("Empty HybridValues:")
hybrid_values_empty.print()

# 2. From VectorValues and DiscreteValues
vec_vals = VectorValues()
vec_vals.insert(X(0), np.array([1.0, 2.0]))
vec_vals.insert(X(1), np.array([3.0]))

disc_vals = DiscreteValues()
disc_vals[D(0)] = 1
disc_vals[D(1)] = 0

hybrid_values_vd = HybridValues(vec_vals, disc_vals)
print("\nHybridValues from VectorValues and DiscreteValues:")
hybrid_values_vd.print()

# 3. From VectorValues, DiscreteValues, and Nonlinear Values
nonlinear_vals = Values()
nonlinear_vals.insert(X(5), Pose2(1, 2, 0.3)) # Example nonlinear type

hybrid_values_all = HybridValues(vec_vals, disc_vals, nonlinear_vals)
print("\nHybridValues from all three types:")
hybrid_values_all.print()

Empty HybridValues:
HybridValues: 
  Continuous: 0 elements
  Discrete: 
  Nonlinear
Values with 0 values:

HybridValues from VectorValues and DiscreteValues:
HybridValues: 
  Continuous: 2 elements
  x0: 1 2
  x1: 3
  Discrete: (d0, 1)(d1, 0)
  Nonlinear
Values with 0 values:

HybridValues from all three types:
HybridValues: 
  Continuous: 2 elements
  x0: 1 2
  x1: 3
  Discrete: (d0, 1)(d1, 0)
  Nonlinear
Values with 1 values:
Value x5: (class gtsam::Pose2)
(1, 2, 0.3)



## Accessing Values

Methods are provided to access the underlying containers and check for key existence.

In [4]:
# Access underlying containers
cont_vals = hybrid_values_all.continuous()
disc_vals_acc = hybrid_values_all.discrete()
nonlin_vals_acc = hybrid_values_all.nonlinear()

print(f"\nAccessed Continuous Values size: {cont_vals.size()}")
print(f"Accessed Discrete Values size: {len(disc_vals_acc)}") # DiscreteValues acts like dict
print(f"Accessed Nonlinear Values size: {nonlin_vals_acc.size()}")

# Check existence
print(f"\nExists Vector X(0)? {hybrid_values_all.existsVector(X(0))}")
print(f"Exists Discrete D(1)? {hybrid_values_all.existsDiscrete(D(1))}")
print(f"Exists Nonlinear X(5)? {hybrid_values_all.existsNonlinear(X(5))}")
print(f"Exists Vector D(0)? {hybrid_values_all.existsVector(D(0))}") # False

# exists() checks across all types (nonlinear checked first)
print(f"Exists X(0)? {hybrid_values_all.exists(X(0))}") # Checks VectorValues
print(f"Exists D(0)? {hybrid_values_all.exists(D(0))}") # Checks DiscreteValues
print(f"Exists X(5)? {hybrid_values_all.exists(X(5))}") # Checks Nonlinear Values

# Access specific values
print(f"\nValue at X(0): {hybrid_values_all.at(X(0))}")
print(f"Value at D(0): {hybrid_values_all.discrete()[D(0)]}")
print(f"Value at X(5): {hybrid_values_all.nonlinear().atPose2(X(5))}") # Use type-specific getter


Accessed Continuous Values size: 2
Accessed Discrete Values size: 2
Accessed Nonlinear Values size: 1

Exists Vector X(0)? True
Exists Discrete D(1)? True
Exists Nonlinear X(5)? True
Exists Vector D(0)? False
Exists X(0)? True
Exists D(0)? True
Exists X(5)? True

Value at X(0): [1. 2.]
Value at D(0): 1
Value at X(5): (1, 2, 0.3)



## Modifying Values (Insert, Update, Retract)

Values can be inserted individually or from other containers. `update` modifies existing keys, while `insert` adds new keys. `retract` applies a delta to the continuous VectorValues part.

In [7]:
hv = HybridValues()

# Insert individual values
hv.insert(X(10), np.array([1.0])) # Vector value
hv.insert(D(10), 1)             # Discrete value

print("After individual inserts:")
hv.print()

# Insert from containers
new_vec = VectorValues()
new_vec.insert(X(12), np.array([5.0]))
new_disc = DiscreteValues()
new_disc[D(11)] = 0

hv.insert(new_vec)
hv.insert(new_disc)
print("\nAfter container inserts:")
hv.print()

# Update existing values
update_vec = VectorValues()
update_vec.insert(X(10), np.array([99.0]))
update_disc = DiscreteValues()
update_disc[D(10)] = 2

hv.update(update_vec)
hv.update(update_disc)
print("\nAfter update:")
hv.print()

# Retract (applies delta to VectorValues part)
delta = VectorValues()
delta.insert(X(10), np.array([-1.0]))
delta.insert(X(12), np.array([0.5]))

hv_retracted = hv.retract(delta)
print("\nAfter retract:")
hv_retracted.print()
print("\nOriginal (unchanged by retract):")
hv.print() # Original is not modified

# Insert or assign
# Replaces if exists, inserts if not.
hv.insert_or_assign(X(10), np.array([100.0])) # Overwrites X(10)
hv.insert_or_assign(D(12), 1) # Inserts D(12)
hv.insert_or_assign(X(13), np.array([13.0])) # Inserts X(13)
print("\nAfter insert_or_assign:")
hv.print()

After individual inserts:
HybridValues: 
  Continuous: 1 elements
  x10: 1
  Discrete: (d10, 1)
  Nonlinear
Values with 0 values:

After container inserts:
HybridValues: 
  Continuous: 2 elements
  x10: 1
  x12: 5
  Discrete: (d10, 1)(d11, 0)
  Nonlinear
Values with 0 values:

After update:
HybridValues: 
  Continuous: 2 elements
  x10: 99
  x12: 5
  Discrete: (d10, 2)(d11, 0)
  Nonlinear
Values with 0 values:

After retract:
HybridValues: 
  Continuous: 2 elements
  x10: 99
  x12: 5
  Discrete: (d10, 2)(d11, 0)
  Nonlinear
Values with 0 values:

Original (unchanged by retract):
HybridValues: 
  Continuous: 2 elements
  x10: 99
  x12: 5
  Discrete: (d10, 2)(d11, 0)
  Nonlinear
Values with 0 values:

After insert_or_assign:
HybridValues: 
  Continuous: 3 elements
  x10: 100
  x12: 5
  x13: 13
  Discrete: (d10, 2)(d11, 0)(d12, 1)
  Nonlinear
Values with 0 values:
